# EMA_short and EMA_long Crossover Backtest

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib as ta
from lightweight_charts import Chart
import asyncio
import nest_asyncio
from websockets import Close
import warnings

# Ignore all warnings from this point forward
warnings.filterwarnings('ignore')

nest_asyncio.apply()

In [2]:
# df = yf.download('TQQQ', start='2010-01-01', multi_level_index=False)
# df.reset_index(inplace=True)
# df.to_csv('TQQQ_data.csv', index=False)
df = pd.read_csv('TQQQ_data.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Close,High,Low,Open,Volume
0,2010-02-11,0.413438,0.415679,0.387651,0.388896,3456000
1,2010-02-12,0.415131,0.418715,0.399848,0.402187,8601600
2,2010-02-16,0.431211,0.432207,0.418217,0.424888,9619200
3,2010-02-17,0.438528,0.438628,0.430414,0.436986,19180800
4,2010-02-18,0.446842,0.449480,0.435442,0.438080,38860800


In [3]:
# Calculate EMA indicators with proper error handling
try:
    df['EMA_12'] = ta.EMA(df['Close'], timeperiod=12)
    df['EMA_26'] = ta.EMA(df['Close'], timeperiod=26)
    
    # More efficient way to calculate crossovers
    df['emabullish'] = (df['EMA_12'] > df['EMA_26']).astype(float)
    df['crossover_EMA'] = df['emabullish'].diff()
    
    # Clean up NaN values
    df = df.dropna().reset_index(drop=True)
    print("EMA calculations completed successfully")
    df.head()
except Exception as e:
    print(f"Error calculating EMAs: {str(e)}")
    raise

EMA calculations completed successfully


In [4]:
def create_chart(df):
    try:
        chart = Chart(title='TQQQ with EMA 12 and EMA 26', maximize=True)
        chart.legend(visible=True)
        chart.set(df)

        # Add EMA lines with improved visibility
        ema12_line = chart.create_line('EMA_12', color='#ffeb3b', width=2, price_label=True)
        ema12_line.set(df[['Date', 'EMA_12']])

        ema26_line = chart.create_line('EMA_26', color='#26c6da', width=2, price_label=True)
        ema26_line.set(df[['Date', 'EMA_26']])
        
        # Create markers for crossover points more efficiently
        markers = []
        crossover_points = df[df['crossover_EMA'] != 0]
        
        for _, row in crossover_points.iterrows():
            if row['crossover_EMA'] == 1:
                markers.append({
                    'time': row['Date'],
                    'position': 'below',
                    'shape': 'arrow_up',
                    'color': '#33de3d',
                    'text': 'Buy'
                })
            elif row['crossover_EMA'] == -1:
                markers.append({
                    'time': row['Date'],
                    'position': 'above',
                    'shape': 'arrow_down',
                    'color': '#f485fb',
                    'text': 'Sell'
                })

        if markers:
            chart.marker_list(markers)

        return chart
    except Exception as e:
        print(f"Error creating chart: {str(e)}")
        raise

if __name__ == '__main__':
    try:
        chart = create_chart(df)
        chart.show(block=True)
    except Exception as e:
        print(f"Error displaying chart: {str(e)}")
        raise

In [5]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import numpy as np

class EMACrossStrategy(Strategy):
    def init(self):
        # Calculate EMAs more efficiently
        price = self.data.Close
        self.ema12 = self.I(ta.EMA, price, 12)
        self.ema26 = self.I(ta.EMA, price, 26)
        
        # Add RSI for overbought/oversold conditions
        self.rsi = self.I(ta.RSI, price, timeperiod=14)
        
    def next(self):
        if len(self.data) < 2:  # Skip first candle
            return
        
        # Calculate position size based on portfolio value (fixed 1.0 = 100% of equity)
        position_size = 1.0  # Use full equity for position
        
        # Check for buy signal with RSI filter
        if (crossover(self.ema12, self.ema26) and 
            self.rsi[-1] < 70):  # Not overbought
            if not self.position:
                self.buy(size=position_size)  # Using fraction of equity instead of absolute units
        
        # Check for sell signal with RSI filter
        elif (crossover(self.ema26, self.ema12) and 
              self.rsi[-1] > 30):  # Not oversold
            if self.position:
                self.position.close()

# Prepare data for backtesting
try:
    bt_data = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']].copy()
    bt_data.set_index('Date', inplace=True)

    # Run backtest with improved parameters
    bt = Backtest(bt_data, EMACrossStrategy,
                  cash=10000,
                  commission=.002,
                  exclusive_orders=True,
                  trade_on_close=False)  # More realistic execution

    # Run the backtest
    results = bt.run()
    
    # Print comprehensive results
    print("\nBacktest Results:")
    print("================")
    print(f"Total Return: {results['Return [%]']:.2f}%")
    print(f"Buy & Hold Return: {results['Buy & Hold Return [%]']:.2f}%")
    print(f"Sharpe Ratio: {results['Sharpe Ratio']:.2f}")
    print(f"Sortino Ratio: {results['Sortino Ratio']:.2f}")
    print(f"Max Drawdown: {results['Max. Drawdown [%]']:.2f}%")
    print(f"# Trades: {results['# Trades']}")
    print(f"Win Rate: {results['Win Rate [%]']:.2f}%")
    print(f"Profit Factor: {results['Profit Factor']:.2f}")
    print(f"Average Trade: {results['Avg. Trade [%]']:.2f}%")
    print(f"Max Trade: {results['Best Trade [%]']:.2f}%")
    print(f"Min Trade: {results['Worst Trade [%]']:.2f}%")

    # Plot the backtest results with additional metrics
    bt.plot(show_legend=True, 
            open_browser=False,
            plot_width=1200,
            plot_equity=True,
            plot_pl=True,
            plot_volume=True,
            plot_drawdown=True)
    
except Exception as e:
    print(f"Error in backtesting: {str(e)}")
    raise

Backtest.run:   0%|          | 0/3895 [00:00<?, ?bar/s]


Backtest Results:
Total Return: 0.85%
Buy & Hold Return: 16365.00%
Sharpe Ratio: 0.38
Sortino Ratio: 0.50
Max Drawdown: -0.41%
# Trades: 57
Win Rate: 43.86%
Profit Factor: 2.33
Average Trade: 4.48%
Max Trade: 79.22%
Min Trade: -18.03%


# Strategy Optimization
Let's optimize the EMA periods to find the best performing combination. We'll test different combinations of fast and slow EMAs.

In [ ]:
class OptimizedEMACrossStrategy(Strategy):
    # Define parameters to optimize with more meaningful ranges
    fast_ema = 12
    slow_ema = 26
    rsi_period = 14
    rsi_overbought = 70
    rsi_oversold = 30
    
    def init(self):
        price = self.data.Close
        self.ema_fast = self.I(ta.EMA, price, self.fast_ema)
        self.ema_slow = self.I(ta.EMA, price, self.slow_ema)
        self.rsi = self.I(ta.RSI, price, timeperiod=self.rsi_period)
    
    def next(self):
        if len(self.data) < 2:
            return
            
        position_size = 1.0  # Use full equity for position
        
        if (crossover(self.ema_fast, self.ema_slow) and 
            self.rsi[-1] < self.rsi_overbought):
            if not self.position:
                self.buy(size=position_size)  # Using fraction of equity
                
        elif (crossover(self.ema_slow, self.ema_fast) and 
              self.rsi[-1] > self.rsi_oversold):
            if self.position:
                self.position.close()

try:
    # Create optimization instance
    bt_opt = Backtest(bt_data, OptimizedEMACrossStrategy,
                      cash=10000,
                      commission=.002,
                      exclusive_orders=True,
                      trade_on_close=False)

    # Run optimization with more focused parameter ranges
    optimization_results = bt_opt.optimize(
        fast_ema=range(8, 21, 1),      # Focus on common fast EMA periods
        slow_ema=range(15, 35, 1),     # Focus on common slow EMA periods
        rsi_period=range(10, 21, 2),   # Common RSI periods
        rsi_overbought=[65, 70, 75, 80],
        rsi_oversold=[20, 25, 30, 35],
        maximize='Sortino Ratio',       # Optimize for risk-adjusted returns
        constraint=lambda p: p.fast_ema < p.slow_ema
    )

    # Get best parameters
    best_params = optimization_results._strategy
    
    print("\nOptimization Results:")
    print("===================")
    print(f"Best Fast EMA Period: {best_params.fast_ema}")
    print(f"Best Slow EMA Period: {best_params.slow_ema}")
    print(f"Best RSI Period: {best_params.rsi_period}")
    print(f"Best RSI Overbought: {best_params.rsi_overbought}")
    print(f"Best RSI Oversold: {best_params.rsi_oversold}")
    print(f"Best Sortino Ratio: {optimization_results['Sortino Ratio']:.2f}")
    print(f"Best Sharpe Ratio: {optimization_results['Sharpe Ratio']:.2f}")
    print(f"Best Total Return: {optimization_results['Return [%]']:.2f}%")
    print(f"Best Win Rate: {optimization_results['Win Rate [%]']:.2f}%")
    print(f"Number of Trades: {optimization_results['# Trades']}")

    # Run final backtest with optimal parameters
    optimal_bt = Backtest(bt_data, OptimizedEMACrossStrategy,
                         cash=10000,
                         commission=.002,
                         exclusive_orders=True,
                         trade_on_close=False)

    optimal_results = optimal_bt.run(
        fast_ema=best_params.fast_ema,
        slow_ema=best_params.slow_ema,
        rsi_period=best_params.rsi_period,
        rsi_overbought=best_params.rsi_overbought,
        rsi_oversold=best_params.rsi_oversold
    )

    # Plot final results
    optimal_bt.plot()

except Exception as e:
    print(f"Error in optimization: {str(e)}")
    raise

# Visualization of Optimization Results
Let's create a heatmap to visualize how different combinations of EMAs perform.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def create_optimization_visualizations(bt_opt, parameter_ranges):
    try:
        # Extract results for visualization
        results_data = []
        
        # Get combinations for main EMA parameters
        for fast_ema in range(8, 21, 1):
            for slow_ema in range(15, 35, 1):
                if fast_ema < slow_ema:
                    result = bt_opt.run(
                        fast_ema=fast_ema,
                        slow_ema=slow_ema,
                        rsi_period=14,  # Use default RSI settings for this analysis
                        rsi_overbought=70,
                        rsi_oversold=30
                    )
                    results_data.append({
                        'Fast EMA': fast_ema,
                        'Slow EMA': slow_ema,
                        'Sortino Ratio': result['Sortino Ratio'],
                        'Sharpe Ratio': result['Sharpe Ratio'],
                        'Return': result['Return [%]'],
                        'Win Rate': result['Win Rate [%]'],
                        'Max Drawdown': result['Max. Drawdown [%]'],
                        'Trades': result['# Trades']
                    })
        
        # Convert to DataFrame
        results_df = pd.DataFrame(results_data)
        
        # Create multiple subplots for different metrics
        fig = plt.figure(figsize=(20, 15))
        gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)
        
        # Plot 1: Sortino Ratio Heatmap
        ax1 = fig.add_subplot(gs[0, 0])
        pivot_sortino = results_df.pivot(index='Slow EMA', columns='Fast EMA', values='Sortino Ratio')
        sns.heatmap(pivot_sortino, cmap='RdYlGn', center=0,
                    annot=True, fmt='.2f',
                    cbar_kws={'label': 'Sortino Ratio'}, ax=ax1)
        ax1.set_title('Sortino Ratio by EMA Combinations')
        
        # Plot 2: Return Heatmap
        ax2 = fig.add_subplot(gs[0, 1])
        pivot_return = results_df.pivot(index='Slow EMA', columns='Fast EMA', values='Return')
        sns.heatmap(pivot_return, cmap='RdYlGn', center=0,
                    annot=True, fmt='.1f',
                    cbar_kws={'label': 'Return %'}, ax=ax2)
        ax2.set_title('Total Return % by EMA Combinations')
        
        # Plot 3: Win Rate vs Number of Trades Scatter
        ax3 = fig.add_subplot(gs[1, 0])
        scatter = ax3.scatter(results_df['Trades'], 
                            results_df['Win Rate'],
                            c=results_df['Return'],
                            cmap='RdYlGn',
                            s=100)
        plt.colorbar(scatter, label='Return %', ax=ax3)
        ax3.set_xlabel('Number of Trades')
        ax3.set_ylabel('Win Rate %')
        ax3.set_title('Win Rate vs Number of Trades')
        
        # Plot 4: Risk-Return Analysis
        ax4 = fig.add_subplot(gs[1, 1])
        scatter = ax4.scatter(results_df['Max Drawdown'].abs(),
                            results_df['Return'],
                            c=results_df['Sortino Ratio'],
                            cmap='RdYlGn',
                            s=100)
        plt.colorbar(scatter, label='Sortino Ratio', ax=ax4)
        ax4.set_xlabel('Maximum Drawdown %')
        ax4.set_ylabel('Total Return %')
        ax4.set_title('Risk-Return Analysis')
        
        plt.suptitle('Strategy Optimization Analysis', size=16, y=1.02)
        plt.tight_layout()
        plt.show()

        # Print top combinations
        print("\nTop 10 Parameter Combinations by Sortino Ratio:")
        print("============================================")
        top_sortino = results_df.nlargest(10, 'Sortino Ratio')
        print(top_sortino.round(2).to_string(index=False))
        
        # Calculate optimal parameter ranges
        print("\nOptimal Parameter Ranges:")
        print("=======================")
        metrics = ['Sortino Ratio', 'Sharpe Ratio', 'Return', 'Win Rate']
        for metric in metrics:
            top_5 = results_df.nlargest(5, metric)
            print(f"\nTop 5 by {metric}:")
            print(f"Fast EMA range: {top_5['Fast EMA'].min()}-{top_5['Fast EMA'].max()}")
            print(f"Slow EMA range: {top_5['Slow EMA'].min()}-{top_5['Slow EMA'].max()}")
            
    except Exception as e:
        print(f"Error in visualization: {str(e)}")
        print("Please ensure optimization has completed successfully.")
        raise

# Execute the visualization
try:
    parameter_ranges = {
        'fast_ema': range(8, 21, 1),
        'slow_ema': range(15, 35, 1)
    }
    create_optimization_visualizations(bt_opt, parameter_ranges)
except Exception as e:
    print(f"Failed to create visualizations: {str(e)}")